<a href="https://colab.research.google.com/github/seongmin0219/AlgorithmDataStructure/blob/master/baseline_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import gc
import pandas as pd
import numpy as np
import gc 
from datetime import datetime
import os
import time
import json
# 아이템 연관도를 봐야되는데
import re
from pathlib import Path
import numba as nb
from tqdm import tqdm

In [3]:
from google.colab import drive
drive.mount('/gdrive')
import os
from pathlib import Path

Mounted at /gdrive


In [4]:
Path('./gdrive/MyDrive/')
file_path = '/gdrive/MyDrive/'

if os.path.exists(file_path):
  print('exists')

exists


In [5]:
DATA_DIR = '/gdrive/MyDrive/data/'

In [6]:
import json
def load_json(fname):
    with open(fname, encoding="utf-8") as f:
        json_obj = json.load(f)
    return json_obj

In [7]:
# Opening JSON file
f = open(DATA_DIR + 'train_user_seq_log.json')

# returns JSON object as
# a dictionary
data = json.load(f)

# Iterating through the jsoa

In [8]:
df = pd.DataFrame(list(data.items()),columns = ['user_id','item_timestamp'])

In [9]:
def generate_train(df):
    user_ids = []
    item_ids = []
    timestamps = []
    for rows in tqdm(df.iterrows()):
        item_timestamp = rows[1]['item_timestamp']
        user_id = rows[1]['user_id']
        for idx in range(len(item_timestamp)):
            item = item_timestamp[idx][0]
            timestamp = item_timestamp[idx][1]
            user_ids.append(user_id)
            item_ids.append(item)
            timestamps.append(timestamp)
    train_df = pd.DataFrame(user_ids,columns=['user_ids'])
    train_df['item_ids'] = item_ids
    train_df['timestamp'] = timestamps
    return train_df

In [10]:
def load_json(fname):
    with open(fname, encoding="utf-8") as f:
        json_obj = json.load(f)
    return json_obj

In [11]:
train_db = generate_train(df)

672116it [00:36, 18475.13it/s]


In [12]:
train_db = train_db.rename(columns={'item_ids': 'item_id', 'user_ids':'user_id'})

[['item_805696', '2021-02-11 13:03:42'],
 ['item_386903', '2021-02-11 13:03:52'],
 ['item_3832', '2021-02-11 13:04:07'],
 ['item_849824', '2021-02-11 13:05:04'],
 ['item_815594', '2021-02-11 13:06:19'],
 ['item_814300', '2021-02-11 13:07:28'],
 ['item_930329', '2021-02-11 13:09:14'],
 ['item_823097', '2021-02-11 13:09:40'],
 ['item_837711', '2021-02-11 13:13:35'],
 ['item_704626', '2021-02-11 13:15:10'],
 ['item_46095', '2021-02-11 13:17:02'],
 ['item_542895', '2021-02-11 13:20:31'],
 ['item_229908', '2021-02-11 13:23:45'],
 ['item_935600', '2021-02-11 13:25:07'],
 ['item_700459', '2021-02-11 13:25:53'],
 ['item_114309', '2021-02-11 13:26:51'],
 ['item_212637', '2021-02-11 21:24:05'],
 ['item_691869', '2021-02-12 00:38:57'],
 ['item_648359', '2021-02-12 00:39:12'],
 ['item_202047', '2021-02-12 07:43:52'],
 ['item_929659', '2021-02-12 07:44:14'],
 ['item_704626', '2021-02-12 07:44:54'],
 ['item_930329', '2021-02-12 07:44:59'],
 ['item_190207', '2021-02-12 07:48:01'],
 ['item_626598', '2

In [13]:
@nb.jit(nopython=True, cache=True)
def get_topk(cnts, topk , k ):
    for item_id , cnt in cnts.items():
        topk[item_id] = np.array(heap_topk(cnt,1,k))

In [14]:
import heapq

In [15]:
@nb.jit(nopython=True , cache=True)
def heap_topk(cnt, overwrite ,cap):
    q = [(0.0, 0, 0) for _ in range(0)]
    for i, (k,n) in enumerate(cnt.items()):
        if overwrite == 1:
            heapq.heappush(q, (n,i,k))
        else:
            heapq.heappush(q, (n,-i,k))
        if len(q) > cap:
            heapq.heappop(q)
    return [heapq.heappop(q)[2] for _ in range(len(q))][::-1]

In [16]:
topk = nb.typed.Dict.empty(
    key_type=nb.types.int64,
    value_type=nb.types.int64[:])
cnt = nb.typed.Dict.empty(key_type=nb.types.int64, value_type=nb.types.float64)
cnts = nb.typed.Dict.empty(
    key_type=nb.types.int64,
    value_type=nb.typeof(nb.typed.Dict.empty(key_type=nb.types.int64, value_type=nb.types.float64)))

In [17]:
train_db['item_id'] = train_db['item_id'].apply(lambda x: np.int32(x.split("_")[-1]) )
train_db['timestamp'] = train_db['timestamp'].apply(lambda x: time.mktime(datetime.strptime(x,"%Y-%m-%d %H:%M:%S").timetuple()) )

In [18]:
# item similarity 로 가중치 줘도 되겠다 time weighted 와 ,
items_timestamps = train_db[['item_id','timestamp']].values

In [20]:
df['length'] = df['item_timestamp'].apply(lambda x : len(x))

df['idx'] = df['length'].cumsum()
df['idx'] = df['idx'].shift(1)
df.iloc[0]['idx'] = 0
df.fillna(0,inplace=True)
df['idx'] = df['idx'].astype(np.int64)
df['user_id'] = df['user_id'].apply(lambda x : np.int32(x.split('_')[-1]))

AttributeError: ignored

In [94]:
# 이게 적절하지 않은건가

'2021-02-11 13:03:42'

'2021-05-09 22:45:46'

In [23]:
@nb.jit(nopython = True, parallel = True, cache = True)
def get_pairs(items_timestamps,row, cnts,par_n):
    pairs = [{(0, 0): 0.0 for _ in range(0)} for _ in range(par_n)] 

    for par_i in nb.prange(par_n):
      # if par_i % 100 ==0:
        # print(par_i)
      user_id,idx, length = row[par_i]
      get_single_pair(pairs[par_i], items_timestamps ,idx, length)
      
    for par_i in range(par_n):
        for (aid1, aid2), w in pairs[par_i].items():
            if aid1 not in cnts: cnts[aid1] = {0: 0.0 for _ in range(0)}
            cnt = cnts[aid1]
            if aid2 not in cnt: cnt[aid2] = 0.0
            cnt[aid2] += w

    # for par_i in range(par_n):
    #     for (aid1, aid2), w in pairs[par_i].items():
    #         if aid1 not in cnts: cnts[aid1] = {0: 0.0 for _ in range(0)}
    #         cnt = cnts[aid1]
    #         if aid2 not in cnt: cnt[aid2] = 0.0
    #         cnt[aid2] += w

# 어떻게 끊느냐도 중요한데 유저 전체를 보고 timestamp 로 끊는다?
# 시퀀스 학습시키는게
@nb.jit(nopython=True , cache=True)
def get_single_pair(pairs, item_timestamps,idx,length):
    TIME_WEIGHT =1
    mode = TIME_WEIGHT
    min_idx = idx
    max_idx = idx+length

    for i in range(min_idx , max_idx):
        for j in range(i+1, max_idx):
            timestamp_i =  item_timestamps[i][1]
            timestamp_j =  item_timestamps[j][1]
            # 하루가 넘어가면 break
            time_diff = timestamp_j - timestamp_i
            if time_diff  >= 24 * 60 * 60: break
            if item_timestamps[i][0] == item_timestamps[j][0]: continue # 두 아이템이 같으면 같은 세션이라도 넘어간다
            # 가까운 시간이라도 전혀 다른 아이템일 수 있다
            if mode == TIME_WEIGHT:
               #나의 임의 시간 weighted
                if 0 <= time_diff <= 300: w = 5
                elif 300< time_diff <= 600: w = 4
                elif 600< time_diff <=1800: w = 3
                elif 1800 < time_diff <=3600: w = 2
                elif 3600 < time_diff <= 3600 * 6: w = 1
                else: w = 0
              # 정규화해서 weight를 적용하는건가
            # start_time은 session이기 때문이고 이건 user 별로 있기 때문에 session으로는 나누지 않는다 , 전체 기간 대비 얼마나 짧은 기간내에 검색으로 이어졌는지로 판단
# w1 = 1 + 3 * (ts[i] + start_time - 1659304800) / (1662328791 - 1659304800)
#                w2 = 1 + 3 * (ts[j] + start_time - 1659304800) / (1662328791 - 1659304800) 각 세션
            pairs[(item_timestamps[i][0], item_timestamps[j][0])] = w
            pairs[(item_timestamps[j][0], item_timestamps[i][0])] = w


In [98]:


start_time = df.iloc[0]['item_timestamp'][0][1]
end_time = df.iloc[0]['item_timestamp'][-1][1]

In [ ]:
test_user_seq_log['timestamp'] = test_user_seq_log['item_timestamp'].apply(lambda x: np.array([ int(time.mktime(datetime.strptime(val[1],"%Y-%m-%d %H:%M:%S").timetuple())) for val in x ]) )

In [ ]:

start_time_s = time.mktime(datetime.strptime(start_time, "%Y-%m-%d %H:%M:%S").timetuple())

In [109]:

end_time_s = time.mktime(datetime.strptime(end_time, "%Y-%m-%d %H:%M:%S").timetuple())

In [116]:
ts_i =time.mktime(datetime.strptime( df.iloc[500]['item_timestamp'][0][1], "%Y-%m-%d %H:%M:%S").timetuple())

In [121]:

datetime.fromtimestamp(1662328791)

datetime.datetime(2022, 9, 4, 21, 59, 51)

In [1]:

datetime.fromtimestamp(1659304800)

NameError: ignored

In [118]:
(ts_i + start_time_s - end_time_s )/ (end_time_s - start_time_s)

212.60009992420274

In [115]:
(ts_i + start_time_s - end_time_s )/ (end_time_s - start_time_s)

212.60004973698722

In [117]:
# 이러니까 너무 작은 차이가 난다 
(ts_i + start_time_s - end_time_s )/ (start_times - start_time_s)



212.60009992420274

In [89]:
+ start_time - end_time)

NameError: ignored

In [75]:
datetime.fromtimestamp(items_timestamps[1][1])

datetime.datetime(2021, 2, 11, 13, 3, 52)

In [79]:
start_time = items_timestamps[0][1]

In [82]:
# 총 시간 간격으로 나누어주면 얼마나 거기서 많은 비율과 값을 차지하는 지로 가중치를 계산하는 듯하다
(items_timestamps[2][1]  + start_time -1659304800) / (1662328791 - 1659304800)


518.1207447376662

In [77]:
datetime.fromtimestamp(1659304800)  

datetime.datetime(2022, 7, 31, 22, 0)

In [78]:
datetime.fromtimestamp(1662328791)

datetime.datetime(2022, 9, 4, 21, 59, 51)

In [74]:
# datetime.strptime(,"%Y-%m-%d %H:%M:%S").timetuple() 

TypeError: ignored

In [69]:
items_timestamps[0][1]

1613048622.0

In [72]:
items_timestamps[2][1]


1613048647.0

array([[8.05696000e+05, 1.61304862e+09],
       [3.86903000e+05, 1.61304863e+09],
       [3.83200000e+03, 1.61304865e+09],
       ...,
       [1.04700000e+03, 1.62082452e+09],
       [7.10805000e+05, 1.62082455e+09],
       [2.32869000e+05, 1.62082460e+09]])

In [24]:
%%time
# 30초에 1프로
# 3000 초 100프로 50분? 갈수록 빨라지는 듯 cache 해서 그런지  1분에 65건 정도니까원래 코드랑비슷하다 내가 이전에 짠거랑 비교하기 작은 차이라도 큰 차이난다 속도가 엄청나게 !!

# cnt저장이 오래걸린다 그래도 할만한 속도로 대폭 줄음
# 빠르고 병렬적인 base line end-to-end를 위ㅎ 
tail = 30
min_idx = 0
# 5개 전까지 확인
length =20
idx = 0
TIME_WEIGHT =1
parallel =1024
# 아니면 top40를 뽑고 거기서 rerank로 섞어주는 게 더 나을 수도 있다
topn = 20
# for start_idx in range(len(grp_df)-5):
n = 200000  #chunk row size
list_df = [train_db[i:i+n] for i in range(0,train_db.shape[0],n)]
chunks_1 = list_df[0]
grp_chunk = chunks_1.groupby('user_id')
# chunk에서 user_id가 짤렸다면 unique user id 보다 길이가 더 늘어날 수 있으므로 len(df)+알파
# 161개의 청크이므로 중복은 161개 정도에서만 발생, 안 발생하도록 코딩하는 게 제일 좋지만
pairs = [{(0,0) : 0.0 for _ in range(0)} for _ in range(len(df)+300)]
total_len = len(df) 
max_idx = len(df)
cnts = nb.typed.Dict.empty(
        key_type = nb.types.int64,
        value_type = nb.typeof(nb.typed.Dict.empty(key_type = nb.types.int64, value_type = nb.types.float64)))
# 확연히 다른 속도.. 일단은 자자 60만행에대해서 각 행 처리한듯하니 옛날같았으면 느리네하고 기다릴텐데

for idx in tqdm(range(0,max_idx, parallel)):
    row = df.iloc[idx:min(idx + parallel, max_idx)][['user_id', 'idx', 'length']].values
    par_n = len(row)
    get_pairs(items_timestamps, row, cnts,par_n)


# get topk from counter
topk = nb.typed.Dict.empty(
        key_type = nb.types.int64,
        value_type = nb.types.int64[:])
get_topk(cnts, topk, topn)

# del cnts; gc.collect() 

  0%|          | 0/657 [00:00<?, ?it/s]<ipython-input-23-249cf2ebb2bc>:9: NumbaTypeSafetyWarning: unsafe cast from UniTuple(float64 x 2) to UniTuple(int64 x 2). Precision may be lost.
  get_single_pair(pairs[par_i], items_timestamps ,idx, length)
100%|██████████| 657/657 [03:13<00:00,  3.39it/s]


CPU times: user 4min 52s, sys: 3.73 s, total: 4min 56s
Wall time: 3min 27s


In [25]:
import numpy as np


def ndcg(recs: dict, k: int):
    '''
    :param recs: reco result and label
        example:
            {user: {'recs': [reco_item1, ..., reco_item@], 'gt': gt_item(== label)}}
    :param k: ndcg@k
    :return: ndcg@k score
    '''

    ndcg_ = 0
    for user, each in recs.items():
        reco_items = each['recs']
        gt_item = each['gt']

        for idx, reco_item in enumerate(reco_items):
            if gt_item == reco_item:
                if idx < k:
                    ndcg_ += 1 / np.log2(idx + 2)
                break

    return float(ndcg_ / len(recs))


def hit(recs: dict, k: int):
    '''
    :param recs: reco result and label
        example:
            {user: {'recs': [reco_item1, ..., reco_item@], 'gt': gt_item(== label)}}
    :param k: hit@k
    :return: hit@k(==recall@k) score
    ''' 
    hit_ = 0
    for user, each in recs.items():
        reco_items = each['recs']
        gt_item = each['gt']

        for idx, reco_item in enumerate(reco_items):
            if gt_item == reco_item:
                if idx < k:
                    hit_ += 1
                break

    return float(hit_ / len(recs))


In [26]:

test_user_label = load_json('/gdrive/MyDrive/data/test_user_label.json')
test_user_seq_log = load_json('/gdrive/MyDrive/data/test_user_seq_log.json')
test_user_label_df = pd.DataFrame(list(test_user_label.items()), columns=['user_id', 'item_tiemstamp'])
test_user_seq_log = pd.DataFrame(list(test_user_seq_log.items()), columns=['user_id', 'item_timestamp'])
test_user_label_df.head()
test_user_seq_log = test_user_seq_log.rename(columns={'text':'item_timestamp'})

In [27]:
@nb.jit(nopython=True, cache=True)
def inference(row, result_clicks,topk,seq_weight):
  for user_id, item_id, timestamp in row:
    unique_aids = nb.typed.Dict.empty(key_type = nb.types.int64 , value_type= nb.types.float64) 
    cnt = nb.typed.Dict.empty(key_type= nb.types.int64, value_type = nb.types.float64)

    # 최신순부터 거꾸로 배열
    new_item_id = item_id[::-1]
    new_timestamp = timestamp[::-1]

    for a in item_id:
    # t = a[0].split("_")[-1]
    # t = int(a[0])
      unique_aids[a] = 0 
    
    if len(unique_aids) >= 20:
      sequence_weight = np.power(2 , np.linspace(seq_weight, 1, len(new_item_id)))[::-1]-1
      for a , w in zip(new_item_id,sequence_weight):
        if a not in cnt:
         cnt[a] = 0
        cnt[a[0]] += w # * 클릭 ,장바구니, 구매에 따른 가중치 추가가능
      result_candidates = heap_topk(cnt, 0, 20)
    else:
      result_candidates = list(unique_aids)
      for a in result_candidates:
        if a not in topk: continue
        for b in topk[a]:
          if b in unique_aids: continue
          if b not in cnt: cnt[b] = 0
          cnt[b] += 1 # 정렬된 topk에서 co-visitation이 높은 아이템을 순서대로 가져와 모자란 길이만큼 채운다
      result_candidates.extend(heap_topk(cnt, 0, 20 - len(result_candidates)))
    result_clicks[user_id] = np.array(result_candidates)

In [28]:
test_user_seq_log['item_id'] = test_user_seq_log['item_timestamp'].apply(lambda x: np.array([ np.int64(val[0].split("_")[-1]) for val in x])  )
test_user_seq_log['timestamp'] = test_user_seq_log['item_timestamp'].apply(lambda x: np.array([ int(time.mktime(datetime.strptime(val[1],"%Y-%m-%d %H:%M:%S").timetuple())) for val in x ]) )

In [29]:
type(test_user_seq_log['item_id'].values[0][0])

numpy.int64

In [26]:
type(test_user_seq_log['timestamp'].values[0][0])


numpy.int64

In [30]:
def generate_train(df):
    user_ids = []
    item_ids = []
    timestamps = []
    for rows in tqdm(df.iterrows()):
        item_timestamp = rows[1]['item_timestamp']
        user_id = rows[1]['user_id']
        for idx in range(len(item_timestamp)):
            item = item_timestamp[idx][0]
            timestamp = item_timestamp[idx][1]
            user_ids.append(user_id)
            item_ids.append(item)
            timestamps.append(timestamp)
    train_df = pd.DataFrame(user_ids,columns=['user_ids'])
    train_df['item_ids'] = item_ids
    train_df['timestamps'] = timestamps
    return train_df

In [31]:

test_db = generate_train(test_user_seq_log)

587486it [00:25, 22627.85it/s]


In [32]:
test_user_seq_log.iloc[0]['idx'] = 0 

In [33]:
test_user_seq_log['length'] = test_user_seq_log['item_timestamp'].apply(lambda x : len(x))

test_user_seq_log['idx'] = test_user_seq_log['length'].cumsum()
test_user_seq_log['idx'] = test_user_seq_log['idx'].shift(1)
test_user_seq_log.fillna(0,inplace=True)
test_user_seq_log['idx'] = test_user_seq_log['idx'].astype(np.int64) 



In [34]:
test_user_seq_log

,user_id,item_timestamp,item_id,timestamp,length,idx
0,user_0,"[[item_766598, 2021-05-05 22:05:21], [item_652...","[766598, 65273, 580437, 773271, 580437, 228561...","[1620252321, 1620252532, 1620252675, 162025294...",20,0
1,user_1,"[[item_183462, 2021-04-30 13:11:22], [item_374...","[183462, 374145, 44563, 204302, 110301, 793635...","[1619788282, 1619788315, 1619788360, 161978848...",20,20
2,user_2,"[[item_108368, 2021-05-11 19:41:28], [item_414...","[108368, 41459, 667880, 869425, 932949, 869425...","[1620762088, 1620766788, 1620766797, 162076680...",20,40
3,user_3,"[[item_520146, 2021-03-13 13:51:46], [item_670...","[520146, 670721, 962828, 560874, 181147, 54449...","[1615643506, 1615935606, 1616058665, 161610918...",20,60
4,user_4,"[[item_577279, 2021-05-06 23:23:03], [item_374...","[577279, 374594, 99709, 382452, 747701, 717701...","[1620343383, 1620343410, 1620343540, 162034357...",20,80
...,...,...,...,...,...,...
587481,user_829386,"[[item_554650, 2021-05-12 12:55:35], [item_810...","[554650, 810147, 813829, 893014, 130000]","[1620824135, 1620824148, 1620824177, 162082420...",5,7025309
587482,user_829390,"[[item_310178, 2021-05-12 12:56:18], [item_866...","[310178, 866254, 945164, 47147, 851941, 712124...","[1620824178, 1620824190, 1620824223, 162082423...",8,7025314
587483,user_829393,"[[item_115233, 2021-05-12 12:56:42], [item_492...","[115233, 492271]","[1620824202, 1620824207]",2,7025322
587484,user_829397,"[[item_503200, 2021-05-12 12:58:08], [item_517...","[503200, 517483]","[1620824288, 1620824438]",2,7025324


In [35]:
type(test_user_seq_log['user_id'].values[0])


str

In [36]:
test_user_seq_log

,user_id,item_timestamp,item_id,timestamp,length,idx
0,user_0,"[[item_766598, 2021-05-05 22:05:21], [item_652...","[766598, 65273, 580437, 773271, 580437, 228561...","[1620252321, 1620252532, 1620252675, 162025294...",20,0
1,user_1,"[[item_183462, 2021-04-30 13:11:22], [item_374...","[183462, 374145, 44563, 204302, 110301, 793635...","[1619788282, 1619788315, 1619788360, 161978848...",20,20
2,user_2,"[[item_108368, 2021-05-11 19:41:28], [item_414...","[108368, 41459, 667880, 869425, 932949, 869425...","[1620762088, 1620766788, 1620766797, 162076680...",20,40
3,user_3,"[[item_520146, 2021-03-13 13:51:46], [item_670...","[520146, 670721, 962828, 560874, 181147, 54449...","[1615643506, 1615935606, 1616058665, 161610918...",20,60
4,user_4,"[[item_577279, 2021-05-06 23:23:03], [item_374...","[577279, 374594, 99709, 382452, 747701, 717701...","[1620343383, 1620343410, 1620343540, 162034357...",20,80
...,...,...,...,...,...,...
587481,user_829386,"[[item_554650, 2021-05-12 12:55:35], [item_810...","[554650, 810147, 813829, 893014, 130000]","[1620824135, 1620824148, 1620824177, 162082420...",5,7025309
587482,user_829390,"[[item_310178, 2021-05-12 12:56:18], [item_866...","[310178, 866254, 945164, 47147, 851941, 712124...","[1620824178, 1620824190, 1620824223, 162082423...",8,7025314
587483,user_829393,"[[item_115233, 2021-05-12 12:56:42], [item_492...","[115233, 492271]","[1620824202, 1620824207]",2,7025322
587484,user_829397,"[[item_503200, 2021-05-12 12:58:08], [item_517...","[503200, 517483]","[1620824288, 1620824438]",2,7025324


In [37]:
test_db = test_db.rename(columns={'user_ids': 'user_id','item_ids':'item_id','timestamps':'timestamp'})

In [38]:
test_db['user_id'] = test_db['user_id'].apply(lambda x : np.int64(x.split("_")[-1]))

In [39]:
test_db['timestamp'] = test_db['timestamp'].apply(lambda x: np.int64(time.mktime(datetime.strptime(x,"%Y-%m-%d %H:%M:%S").timetuple()) ))

In [40]:
test_db['item_id'] = test_db['item_id'].apply(lambda x : np.int64(x.split("_")[-1]))

In [49]:
test_user_seq_log['user_id'] = test_user_seq_log['user_id'].apply(lambda x : np.int64(x.split("_")[-1]))

In [42]:
test_items_timestamps =  test_db[['item_id','timestamp']].values

In [43]:
test_user_seq_log

,user_id,item_timestamp,item_id,timestamp,length,idx
0,user_0,"[[item_766598, 2021-05-05 22:05:21], [item_652...","[766598, 65273, 580437, 773271, 580437, 228561...","[1620252321, 1620252532, 1620252675, 162025294...",20,0
1,user_1,"[[item_183462, 2021-04-30 13:11:22], [item_374...","[183462, 374145, 44563, 204302, 110301, 793635...","[1619788282, 1619788315, 1619788360, 161978848...",20,20
2,user_2,"[[item_108368, 2021-05-11 19:41:28], [item_414...","[108368, 41459, 667880, 869425, 932949, 869425...","[1620762088, 1620766788, 1620766797, 162076680...",20,40
3,user_3,"[[item_520146, 2021-03-13 13:51:46], [item_670...","[520146, 670721, 962828, 560874, 181147, 54449...","[1615643506, 1615935606, 1616058665, 161610918...",20,60
4,user_4,"[[item_577279, 2021-05-06 23:23:03], [item_374...","[577279, 374594, 99709, 382452, 747701, 717701...","[1620343383, 1620343410, 1620343540, 162034357...",20,80
...,...,...,...,...,...,...
587481,user_829386,"[[item_554650, 2021-05-12 12:55:35], [item_810...","[554650, 810147, 813829, 893014, 130000]","[1620824135, 1620824148, 1620824177, 162082420...",5,7025309
587482,user_829390,"[[item_310178, 2021-05-12 12:56:18], [item_866...","[310178, 866254, 945164, 47147, 851941, 712124...","[1620824178, 1620824190, 1620824223, 162082423...",8,7025314
587483,user_829393,"[[item_115233, 2021-05-12 12:56:42], [item_492...","[115233, 492271]","[1620824202, 1620824207]",2,7025322
587484,user_829397,"[[item_503200, 2021-05-12 12:58:08], [item_517...","[503200, 517483]","[1620824288, 1620824438]",2,7025324


In [54]:
@nb.jit(nopython=True, cache=True)
def inference(row, result_clicks,topk,seq_weight):
  for user_id, idx, length in row:
    unique_aids = nb.typed.Dict.empty(key_type = nb.types.int64 , value_type= nb.types.float64) 
    cnt = nb.typed.Dict.empty(key_type= nb.types.int64, value_type = nb.types.float64)

    # 최신순부터 거꾸로 배열
    #new_item_id = item_id[::-1]
    #new_timestamp = timestamp[::-1]
    
    candidates = test_items_timestamps[idx:idx + length][::-1]

    # for a in candidates: 
    #   unique_aids[a[0]] = 0.0
    
    # if len(unique_aids) >= 20:
    #   sequence_weight = np.power(2 , np.linspace(seq_weight, 1, len(candidates)))[::-1]-1
    for a in candidates:
      if a[0] not in cnt:
        cnt[a[0]] = 0.0
      #cnt[a[0]] += w # * 클릭 ,장바구니, 구매에 따른 가중치 추가가능

    result_candidates = heap_topk(cnt, 0, len(candidates))
    # else:
      # result_candidates = list(unique_aids)
      # for a in result_candidates:
      #   if a not in topk: continue
      #   for b in topk[a]:
      #     if b in unique_aids: continue
      #     if b not in cnt: cnt[b] = 0.0
      #     cnt[b] += 1 # 정렬된 topk에서 co-visitation이 높은 아이템을 순서대로 가져와 모자란 길이만큼 채운다
      # result_candidates.extend(heap_topk(cnt, 0, 20 - len(result_candidates)))
    result_clicks[user_id] = np.array(result_candidates)





In [45]:
#result place holder
result_clicks = nb.typed.Dict.empty(
    key_type = nb.types.int64,
    value_type = nb.types.int64[:])
result_buy = nb.typed.Dict.empty(
    key_type = nb.types.int64,
    value_type = nb.types.int64[:]) 

In [55]:
for idx in tqdm(range(0, len(test_user_seq_log), parallel)):
  row = test_user_seq_log.iloc[idx:min(idx+parallel , len(test_user_seq_log))][['user_id','idx','length']].values
  inference(row , result_clicks,topk,0.1)

  0%|          | 0/574 [00:00<?, ?it/s]<ipython-input-54-3be8f913b9d8>:1: NumbaWarning: Cannot cache compiled function "inference" as it uses dynamic globals (such as ctypes pointers and large global arrays)
  @nb.jit(nopython=True, cache=True)
100%|██████████| 574/574 [00:02<00:00, 229.42it/s]


In [58]:
test_user_seq_log

,user_id,item_timestamp,item_id,timestamp,length,idx
0,0,"[[item_766598, 2021-05-05 22:05:21], [item_652...","[766598, 65273, 580437, 773271, 580437, 228561...","[1620252321, 1620252532, 1620252675, 162025294...",20,0
1,1,"[[item_183462, 2021-04-30 13:11:22], [item_374...","[183462, 374145, 44563, 204302, 110301, 793635...","[1619788282, 1619788315, 1619788360, 161978848...",20,20
2,2,"[[item_108368, 2021-05-11 19:41:28], [item_414...","[108368, 41459, 667880, 869425, 932949, 869425...","[1620762088, 1620766788, 1620766797, 162076680...",20,40
3,3,"[[item_520146, 2021-03-13 13:51:46], [item_670...","[520146, 670721, 962828, 560874, 181147, 54449...","[1615643506, 1615935606, 1616058665, 161610918...",20,60
4,4,"[[item_577279, 2021-05-06 23:23:03], [item_374...","[577279, 374594, 99709, 382452, 747701, 717701...","[1620343383, 1620343410, 1620343540, 162034357...",20,80
...,...,...,...,...,...,...
587481,829386,"[[item_554650, 2021-05-12 12:55:35], [item_810...","[554650, 810147, 813829, 893014, 130000]","[1620824135, 1620824148, 1620824177, 162082420...",5,7025309
587482,829390,"[[item_310178, 2021-05-12 12:56:18], [item_866...","[310178, 866254, 945164, 47147, 851941, 712124...","[1620824178, 1620824190, 1620824223, 162082423...",8,7025314
587483,829393,"[[item_115233, 2021-05-12 12:56:42], [item_492...","[115233, 492271]","[1620824202, 1620824207]",2,7025322
587484,829397,"[[item_503200, 2021-05-12 12:58:08], [item_517...","[503200, 517483]","[1620824288, 1620824438]",2,7025324


In [59]:
result_clicks[829403]

array([567048, 537578, 628847, 335217, 263005, 167515, 292633, 183824])

In [53]:

# 한 아이템에 이 데이터들 저장하는게 5분이면 아주 빠르다 굿 안알려진거라 공유 받은 이거 나만 잘쓰고 있는 걸 수도 드디어 해냈따 자료구조는 컴퓨터과학적으로 생각하고 꼼꼼함을 곁들여서 정성적으로
cnts[710805]

DictType[int64,float64]<iv=None>({205108: 974.0, 911302: 24.0, 27526: 434.0, 561965: 24.0, 716642: 66.0, 247703: 70.0, 180739: 40.0, 775306: 99.0, 852611: 108.0, 267761: 19.0, 753713: 506.0, 35296: 1097.0, 310343: 214.0, 487164: 21.0, 129407: 47.0, 102468: 65.0, 883209: 31.0, 556108: 2.0, 153293: 25.0, 399654: 501.0, 569886: 233.0, 146176: 67.0, 764004: 1382.0, 885499: 1456.0, 396738: 458.0, 319370: 433.0, 490906: 3318.0, 424308: 2440.0, 204302: 54.0, 344325: 5.0, 163988: 713.0, 31473: 284.0, 385955: 183.0, 720743: 461.0, 114789: 1284.0, 268392: 36.0, 364964: 56.0, 383142: 10.0, 731240: 6.0, 23674: 68.0, 819060: 62.0, 229779: 58.0, 53796: 47.0, 26553: 24.0, 381746: 24.0, 108179: 13.0, 366524: 34.0, 426840: 201.0, 825840: 142.0, 19529: 349.0, 466540: 125.0, 716120: 8.0, 743305: 145.0, 931859: 30.0, 379100: 13.0, 910858: 12.0, 35791: 4.0, 810648: 2.0, 363368: 170.0, 199691: 53.0, 194914: 423.0, 88696: 104.0, 479021: 49.0, 371235: 578.0, 416605: 65.0, 298705: 35.0, 32355: 76.0, 937224: 12

In [64]:

item_names_table = load_json('/gdrive/MyDrive/data/item_names_table.json')
item_df = pd.DataFrame(list(item_names_table.items()),columns = ['item_id','text'])
item_df['text_re'] = [re.sub('[^A-Za-z0-9가-힣]', ' ', s) for s in item_df['text']]
item_df.memory_usage()

Index          128
item_id    7761960
text       7761960
text_re    7761960
dtype: int64

In [65]:
item_df


,item_id,text,text_re
0,item_0,DA 남자 빅사이즈 절개 맨투맨 절개 트레이닝팬츠 SET,DA 남자 빅사이즈 절개 맨투맨 절개 트레이닝팬츠 SET
1,item_1,텐피엠 [카시오] MTP-V005L 남성가죽시계+ 블랙 나토밴드.,텐피엠 카시오 MTP V005L 남성가죽시계 블랙 나토밴드
2,item_2,[바보사랑]여성 옥스퍼드 로퍼-(주)태블리스,바보사랑 여성 옥스퍼드 로퍼 주 태블리스
3,item_3,[해외][블랙모어스] 리프리놀/ 초록홍합 100캡슐 X 3,해외 블랙모어스 리프리놀 초록홍합 100캡슐 X 3
4,item_4,[백화점 SAY][크로커다일]경량덕 다운 점퍼(CL9WDW901),백화점 SAY 크로커다일 경량덕 다운 점퍼 CL9WDW901
...,...,...,...
970240,item_970240,접이식 벤치의자 스툴 스탠드 의자 홈바 인테리어 카페,접이식 벤치의자 스툴 스탠드 의자 홈바 인테리어 카페
970241,item_970241,[오너클랜]기획 신발정리대 소형 6단 1P 현관신발장 오픈신발장,오너클랜 기획 신발정리대 소형 6단 1P 현관신발장 오픈신발장
970242,item_970242,[피우다][피우다 (PIUDA)] 여름바다 상자 3p 세트 (수납함/수납,피우다 피우다 PIUDA 여름바다 상자 3p 세트 수납함 수납
970243,item_970243,[지큐][지큐/쿨제이][빅사이즈 추가] 모자이크 롱 숄 가디건,지큐 지큐 쿨제이 빅사이즈 추가 모자이크 롱 숄 가디건


In [ ]:
np.linspace(0.1 , 1 , 20)

array([0.1       , 0.14736842, 0.19473684, 0.24210526, 0.28947368,
       0.33684211, 0.38421053, 0.43157895, 0.47894737, 0.52631579,
       0.57368421, 0.62105263, 0.66842105, 0.71578947, 0.76315789,
       0.81052632, 0.85789474, 0.90526316, 0.95263158, 1.        ])

In [ ]:
# sequence_weight
np.power(2, np.linspace(0.1 , 1 , 20))[::-1] -1

array([1.        , 0.93539975, 0.87288609, 0.81239163, 0.75385116,
       0.69720154, 0.64238172, 0.58933258, 0.53799694, 0.48831944,
       0.44024654, 0.39372639, 0.34870885, 0.30514539, 0.26298903,
       0.22219432, 0.18271729, 0.14451537, 0.10754738, 0.07177346])

In [60]:
recs ={}
# del cnts
# del test_user_label
# del topk
# 만들기 전 메모리 관리 
for user in test_user_seq_log['user_id'].values: 
  str_user = 'user_'+str(user)
  recs[str_user] = {'recs' :list(map(lambda x: 'item_'+str(x), result_clicks[user])), 'gt' : test_user_label[str_user][0] }
hit_20  = hit(recs,20)

In [61]:
# no time-weighted only last 20 
# 0.16 , only popularity
hit_20

0.16606863823137913

In [62]:
# no time-weighted only last 20 
# 0.16 , only popularity
hit_10  = hit(recs,10)

In [63]:
hit_10

0.15580796818988027

In [56]:
# 0.24 , baseline metric 의 경우에는 hit_10이 0.19로 많이 떨어지는 편이다
# time weighted 는 
# if 0 <= time_diff <= 300: w = 5
# elif 300< time_diff <= 600: w = 4
# elif 600< time_diff <=1800: w = 3
# elif 1800 < time_diff <=3600: w = 2
# elif 3600 < time_diff <= 3600 * 6: w = 1 이렇게 줬을 때의 결과값

hit_20

0.242431309001406

In [57]:
hit_1  = hit(recs,1)

In [58]:
hit_1

0.0

In [59]:
ndcg_10  = ndcg(recs,10)

In [60]:
ndcg_10

0.09579339900110684